## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [ ]:
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten, Dropout, Dense
from keras.optimizers import Adam, RMSprop
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet_v2 import ResNet50V2
import numpy as np
import os

import tensorflow as tf
#gpu_devices = tf.config.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(gpu_devices[0], True)

Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# Training parameters

# 資料路徑
DATASET_PATH  = 'data'

# 影像大小
IMAGE_SIZE = (224, 224)

# 影像類別數
NUM_CLASSES = 10

# 若 GPU 記憶體不足，可調降 batch size 或凍結更多層網路
BATCH_SIZE = 128

# Epoch 數
NUM_EPOCHS = 50

data_augmentation = True

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

In [ ]:
# Load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 6s 0us/step


In [ ]:
# Input image dimensions.
input_shape = x_train.shape[1:]

# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# If subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 1)


In [ ]:
def lr_schedule(epoch):
    """Learning Rate Schedule
    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 40:
        lr *= 1e-3
    elif epoch > 30:
        lr *= 1e-2
    elif epoch > 20:
        lr *= 1e-1
    elif epoch > 10:
        lr *= 3e-1
    print('Learning rate: ', lr)
    return lr

In [ ]:
class MyCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    print(K.eval(self.model.optimizer.lr))

In [ ]:
# Prepare model model saving directory.
save_dir = '/content/drive/My Drive/MyPy' #os.path.join(os.getcwd(), 'saved_models')
model_name = f'cifar10_resnet50-final_{NUM_EPOCHS:03d}.h5'

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=3,
                               min_lr=1e-8)

callbacks = [checkpoint, lr_reducer, lr_scheduler, MyCallback()]

In [ ]:
# 以訓練好的 ResNet50 為基礎來建立模型，
# 捨棄 ResNet50 頂層的 fully connected layers
net = ResNet50(include_top=True, 
               weights='imagenet', 
               input_tensor=None,
               input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))

# remove classification layer
net.layers.pop()
net = Model(inputs=net.input, outputs=net.layers[-1].output)

# 設定凍結與要進行訓練的網路層
for layer in net.layers[:-5]:
    layer.trainable = False

x = net.output
#x = Flatten()(x)

# 增加 DropOut layer
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
# 增加 Dense layer，以 softmax 產生個類別的機率值
output_layer = Dense(NUM_CLASSES, activation='softmax', kernel_initializer='he_normal', name="classifier")(x)

net_final = Model(inputs=net.input, outputs=output_layer)

# 使用 Adam optimizer，以較低的 learning rate 進行 fine-tuning
net_final.compile(optimizer=RMSprop(learning_rate=lr_schedule(0)),
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])

# 輸出整個網路結構
print(net_final.summary())

102858752/102853048 [==============================] - 4s 0us/step
Learning rate:  0.001
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___

In [ ]:
# Run training, with or without data augmentation.
if not data_augmentation:
    print('Not using data augmentation.')
    net_final.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=NUM_EPOCHS,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    # 透過 data augmentation 產生訓練與驗證用的影像資料
    train_datagen = ImageDataGenerator(rotation_range=20,
                                       width_shift_range=0.1,
                                       height_shift_range=0.1,
                                       shear_range=0.1,
                                       zoom_range=0.1,
                                       channel_shift_range=0,
                                       horizontal_flip=True,
                                       fill_mode='nearest')
    train_batches = train_datagen.flow_from_directory(DATASET_PATH + '/train_org',
                                                      target_size=IMAGE_SIZE,
                                                      interpolation='bicubic',
                                                      class_mode='categorical',
                                                      shuffle=True,
                                                      batch_size=BATCH_SIZE)

    valid_datagen = ImageDataGenerator()
    valid_batches = valid_datagen.flow_from_directory(DATASET_PATH + '/valid_org',
                                                      target_size=IMAGE_SIZE,
                                                      interpolation='bicubic',
                                                      class_mode='categorical',
                                                      shuffle=False,
                                                      batch_size=BATCH_SIZE)

    # 輸出各類別的索引值
    for cls, idx in train_batches.class_indices.items():
        print('Class #{} = {}'.format(idx, cls))


    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    train_datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    # 訓練模型
    net_final.fit_generator(train_batches,
                            steps_per_epoch = train_batches.samples // BATCH_SIZE,
                            validation_data = valid_batches,
                            validation_steps = valid_batches.samples // BATCH_SIZE,
                            epochs = NUM_EPOCHS, 
                            verbose=1,
                            callbacks=callbacks)
    
#     net_final.fit_generator(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE),
#                         validation_data=(x_test, y_test),
#                         epochs=NUM_EPOCHS, verbose=1,   
#                         callbacks=callbacks)
    
# Score trained model.
scores = net_final.evaluate_generator(valid_batches, valid_batches.samples // BATCH_SIZE, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Using real-time data augmentation.
Found 50000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.
Class #0 = [0]
Class #1 = [1]
Class #2 = [2]
Class #3 = [3]
Class #4 = [4]
Class #5 = [5]
Class #6 = [6]
Class #7 = [7]
Class #8 = [8]
Class #9 = [9]
Epoch 1/50
Learning rate:  0.001
390/390 [==============================] - 845s 2s/step - loss: 0.9363 - accuracy: 0.6925 - val_loss: 0.1668 - val_accuracy: 0.6550

Epoch 00001: val_accuracy improved from -inf to 0.65505, saving model to /content/drive/My Drive/MyPy/cifar10_resnet50-final_050.h5
0.001
Epoch 2/50
Learning rate:  0.001
390/390 [==============================] - 820s 2s/step - loss: 0.6739 - accuracy: 0.7651 - val_loss: 0.2302 - val_accuracy: 0.6617

Epoch 00002: val_accuracy improved from 0.65505 to 0.66167, saving model to /content/drive/My Drive/MyPy/cifar10_resnet50-final_050.h5
0.001
Epoch 3/50
Learning rate:  0.001
390/390 [==============================] - 817s 2s/step - loss: 0.6163 - accuracy: 

In [ ]:
'''
0 : airplain (飛機)
1 : automobile (汽車)
2 : bird (鳥)
3 : cat (貓)
4 : deer (鹿)
5 : dog (狗)
6 : frog (青蛙)
7 : horse (馬)
8 : ship (船)
9 : truck (卡車)
'''    
def array_to_image(X, y, path="train", resize=False, imgsize=(224,224)):
    from PIL import Image
    import numpy as np
    import os

    base_path = os.path.join(os.getcwd(), DATASET_PATH)
    # 檢查路徑是否存在
    if not os.path.exists(base_path):
        os.mkdir(base_path)
        print("creat dir", base_path)
    
    path = os.path.join(base_path, path)        
    # 檢查路徑是否存在
    if not os.path.exists(path):
        os.mkdir(path)
        print("creat dir", path)
    
    if len(X) != len(y):
        print("data Mismatch")
        return

    for i in range(0, len(X)):
        filepath = os.path.join(path, str(y[i]))
        if not os.path.exists(filepath):
            os.mkdir(filepath)
            print("creat dir", filepath)
            
        # 將圖片使用 BICUBIC 方式延伸到 224 x 224
        img = X[i]
        img = Image.fromarray(img)
        if resize:
            img = img.resize(imgsize, Image.BICUBIC)
        filename = os.path.join(filepath, str(i)+".png")
        img.save(filename)
        #print("save", filename)
            
    return

In [ ]:
#array_to_image(x_train, y_train, "train", True, IMAGE_SIZE)
array_to_image(x_train, y_train, "train_org")

creat dir /content/data
creat dir /content/data/train_org
creat dir /content/data/train_org/[6]
creat dir /content/data/train_org/[9]
creat dir /content/data/train_org/[4]
creat dir /content/data/train_org/[1]
creat dir /content/data/train_org/[2]
creat dir /content/data/train_org/[7]
creat dir /content/data/train_org/[8]
creat dir /content/data/train_org/[3]
creat dir /content/data/train_org/[5]
creat dir /content/data/train_org/[0]


In [ ]:
#array_to_image(x_test, y_test, "valid", True, IMAGE_SIZE)
array_to_image(x_test, y_test, "valid_org")

creat dir /content/data/valid_org
creat dir /content/data/valid_org/[3]
creat dir /content/data/valid_org/[8]
creat dir /content/data/valid_org/[0]
creat dir /content/data/valid_org/[6]
creat dir /content/data/valid_org/[1]
creat dir /content/data/valid_org/[9]
creat dir /content/data/valid_org/[5]
creat dir /content/data/valid_org/[7]
creat dir /content/data/valid_org/[4]
creat dir /content/data/valid_org/[2]
